In [20]:
from pyspark import SparkContext
from datetime import timedelta
from math import sin,cos,sqrt,atan2,radians
import datetime

sc = SparkContext.getOrCreate()
depremRDD = sc.textFile("/home/ercan/Desktop/hadoop/datasets/DepremVerileri-2019-Nisan.txt")
depremRDD = depremRDD.filter(lambda line : "Deprem Kodu" not in line)

def splitAndSelect(line):
    arr = line.split("\t")
    lat = float(arr[4])
    lot = float(arr[5])
    date= arr[2]
    time = arr[3]
    xM=float (arr[7])
    place=(arr[14])
    return (lat,lot,date,time,xM,place)

depremRDD = depremRDD.map(lambda line : splitAndSelect(line))
rdd1=depremRDD.filter(lambda x:x [2]>"1989.12.31" and x[2]<"2019.01.02")
rdd2=rdd1.map(lambda x : (x[0],x[1],(x[2]+" "+x[3]),x[4],x[5]))
rdd2=rdd2.sortBy(lambda x: x [3], ascending =False)
rdd3=sc.parallelize(rdd2.take(10))

def date_time(deprem):
    datetimestr= deprem[2]
    datetimeobj = datetime.datetime.strptime(datetimestr,'%Y.%m.%d %H:%M:%S.%f')
    
    if datetimeobj-timedelta(hours=24) <= datetimeobj <= datetimeobj + timedelta(hours=24):
        return True
    else:
        return False
    
rdd4=rdd2.filter(lambda x : date_time(x)==True)

carDeprem = rdd3.cartesian(rdd4)

def cal_dis(loc_1,loc_2):
    R=6373.0
    
    lat1=radians(loc_1[0])
    lon1=radians(loc_1[1])
    lat2=radians(loc_2[0])
    lon2=radians(loc_2[1])
    
    dlon = lon2-lon1
    dlat = lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a),sqrt(1-a))
    distance=R*c
    return distance

disDeprem = carDeprem.map(lambda x:((x[0][4]+" - "+x[1][4]), x[0][2],"xM=",x[1][3],"distance=",(cal_dis(x[0],x[1]))))
resultDeprem=disDeprem.filter(lambda x:x [3]<20)
resultDeprem.take(10)


[('BASISKELE (KOCAELI) [North East  2.0 km] - BASISKELE (KOCAELI) [North East  2.0 km]',
  '1999.08.17 00:01:37.60',
  'xM=',
  7.4,
  'distance=',
  0.0),
 ('BASISKELE (KOCAELI) [North East  2.0 km] - YEMLICE- (VAN) [North West  1.5 km]',
  '1999.08.17 00:01:37.60',
  'xM=',
  7.2,
  'distance=',
  1170.5167637727886),
 ('BASISKELE (KOCAELI) [North East  2.0 km] - UGUR- (DUZCE) [North East  0.3 km]',
  '1999.08.17 00:01:37.60',
  'xM=',
  7.2,
  'distance=',
  104.50995375799918),
 ('BASISKELE (KOCAELI) [North East  2.0 km] - GUNEBAKAN- (ERZINCAN) [South West  1.7 km]',
  '1999.08.17 00:01:37.60',
  'xM=',
  6.8,
  'distance=',
  827.8795667148048),
 ('BASISKELE (KOCAELI) [North East  2.0 km] - GOKOVA KORFEZI (AKDENIZ)',
  '1999.08.17 00:01:37.60',
  'xM=',
  6.6,
  'distance=',
  476.49275857848426),
 ('BASISKELE (KOCAELI) [North East  2.0 km] - AKDENIZ',
  '1999.08.17 00:01:37.60',
  'xM=',
  6.4,
  'distance=',
  581.6017679040067),
 ('BASISKELE (KOCAELI) [North East  2.0 km] - KUR